In [17]:
import json
import requests
import glob
import subprocess
import main

In [18]:
request = requests.get(f'https://adastra.autosome.ru/api/v4/search/snps/eqtl_gene_id/ENSG00000101076')

In [25]:
eqtls = json.loads(request.text)['results']

window_halfwidth = 2000
eqtl_tuples = []
for eqtl in eqtls:
    eqtl_tuples.append((eqtl['chromosome'],
                        eqtl['position'] - 1 - window_halfwidth,
                        eqtl['position'] + window_halfwidth))
#print(eqtl_tuples)
main.write_tuples_in_bed(eqtl_tuples, "HNF4A-eqtl.bed")
eqtl_tuples

[('chr20', 44409210, 44413211),
 ('chr20', 44409496, 44413497),
 ('chr20', 44409647, 44413648),
 ('chr20', 44409552, 44413553)]

In [23]:
def find_intersections_tsv_eqtl(bed_files, keep_counts=None):
    if keep_counts != [] and keep_counts is not None:
        raise ValueError("keep_counts should be empty list or None")
    if keep_counts is None:
        keep_counts = []

    other_bed = "HNF4A-eqtl.bed"

    intersection_counts = {}
    for i, bed_filename in enumerate(bed_files):
        if i % 50 == 0:
            print(f"Doing {i}/{len(bed_files)}: {bed_filename}")
        intersection_filename = "inters_w_hnf4a_eqtl/with_" + bed_filename.split('/')[-1]
        #print(intersection_filename)
        with open(intersection_filename, "bw") as output_file:
            #print(["bedtools", "intersect", "-a", cistrome_bed, "-b", bed_filename])
            intersection = subprocess.run(["bedtools", "intersect", "-a", other_bed, "-b", bed_filename], capture_output=True).stdout
            output_file.write(intersection)
            #print(intersection, intersection.split(b'\n'))
            
            tf_name = bed_filename.split('/')[-1].split('.')[0]
            keep_counts.append((tf_name, len(intersection.split(b'\n')) - 1))

    print(f"{sum(count[1] for count in keep_counts)} intersections found")

In [26]:
bed_files = glob.glob("adastra_snps/*")
intersection_counts = []
find_intersections_tsv_eqtl(bed_files, keep_counts=intersection_counts)

Doing 0/1140: adastra_snps/ZN443_HUMAN.bed
Doing 50/1140: adastra_snps/STA5B_HUMAN.bed
Doing 100/1140: adastra_snps/ZN776_HUMAN.bed
Doing 150/1140: adastra_snps/HXC5_HUMAN.bed
Doing 200/1140: adastra_snps/PRDM2_HUMAN.bed
Doing 250/1140: adastra_snps/MITF_HUMAN.bed
Doing 300/1140: adastra_snps/FOXO1_HUMAN.bed
Doing 350/1140: adastra_snps/PPARG_HUMAN.bed
Doing 400/1140: adastra_snps/CBP_HUMAN.bed
Doing 450/1140: adastra_snps/RING1_HUMAN.bed
Doing 500/1140: adastra_snps/CCAR2_HUMAN.bed
Doing 550/1140: adastra_snps/PMEPA_HUMAN.bed
Doing 600/1140: adastra_snps/RUNX2_HUMAN.bed
Doing 650/1140: adastra_snps/ZN619_HUMAN.bed
Doing 700/1140: adastra_snps/MYRF_HUMAN.bed
Doing 750/1140: adastra_snps/ZN629_HUMAN.bed
Doing 800/1140: adastra_snps/PBX4_HUMAN.bed
Doing 850/1140: adastra_snps/ZN513_HUMAN.bed
Doing 900/1140: adastra_snps/ZN662_HUMAN.bed
Doing 950/1140: adastra_snps/INO80_HUMAN.bed
Doing 1000/1140: adastra_snps/ZN770_HUMAN.bed
Doing 1050/1140: adastra_snps/THA_HUMAN.bed
Doing 1100/1140: ad